In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [12]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


In [13]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [14]:
use_cuda =True
if use_cuda:
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
batch_size=128
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('../data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset2,batch_size=batch_size)

model = Net().to(device)

optimizer = optim.Adadelta(model.parameters(), lr=0.01)

for epoch in range(1, 10):
    print('Epoch:'+str(epoch))
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch:1

Test set: Average loss: 0.3556, Accuracy: 8981/10000 (90%)

Epoch:2

Test set: Average loss: 0.2557, Accuracy: 9261/10000 (93%)

Epoch:3

Test set: Average loss: 0.2162, Accuracy: 9356/10000 (94%)

Epoch:4

Test set: Average loss: 0.1906, Accuracy: 9434/10000 (94%)

Epoch:5

Test set: Average loss: 0.1690, Accuracy: 9494/10000 (95%)

Epoch:6

Test set: Average loss: 0.1528, Accuracy: 9530/10000 (95%)

Epoch:7

Test set: Average loss: 0.1395, Accuracy: 9573/10000 (96%)

Epoch:8

Test set: Average loss: 0.1271, Accuracy: 9606/10000 (96%)

Epoch:9

Test set: Average loss: 0.1178, Accuracy: 9620/10000 (96%)

